In [36]:
import numpy as np 
import pandas as pd 
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
# import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

# example of loading the inception v3 model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input

# load model
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# summarize the model


In [37]:
batch_size = 40
img_height = 224
img_width = 224


In [38]:
## loading training data
training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './dataset',
    validation_split=0.2,
    subset= "training",
    seed=42,
    image_size= (img_height, img_width),
    batch_size=batch_size
)

Found 3833 files belonging to 2 classes.
Using 3067 files for training.


In [40]:
class_names = training_ds.class_names

In [42]:
## Configuring dataset for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
training_ds = training_ds.cache().prefetch(buffer_size=AUTOTUNE)
testing_ds = testing_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [43]:
## lets define our CNN
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [44]:
model = Model(inputs=baseModel.input, outputs=headModel)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [45]:
## lets train our CNN
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
retVal = model.fit(training_ds, validation_data= testing_ds, epochs = 15)
model.save('model.h5')

Epoch 1/15
77/77 [==============================] - 22s 218ms/step - loss: 0.0906 - accuracy: 0.9700 - val_loss: 26.7773 - val_accuracy: 0.5248
Epoch 2/15
77/77 [==============================] - 14s 188ms/step - loss: 0.0402 - accuracy: 0.9892 - val_loss: 15.5578 - val_accuracy: 0.5352
Epoch 3/15
77/77 [==============================] - 15s 189ms/step - loss: 0.0296 - accuracy: 0.9899 - val_loss: 6.6874 - val_accuracy: 0.7206
Epoch 4/15
77/77 [==============================] - 14s 188ms/step - loss: 0.0405 - accuracy: 0.9866 - val_loss: 23.6205 - val_accuracy: 0.4961
Epoch 5/15
77/77 [==============================] - 15s 190ms/step - loss: 0.0197 - accuracy: 0.9961 - val_loss: 15.7667 - val_accuracy: 0.5836
Epoch 6/15
77/77 [==============================] - 15s 194ms/step - loss: 0.0475 - accuracy: 0.9860 - val_loss: 5.6349 - val_accuracy: 0.5614
Epoch 7/15
77/77 [==============================] - 24s 314ms/step - loss: 0.0287 - accuracy: 0.9912 - val_loss: 2.6629 - val_accuracy: 0.

C:\Users\laimo\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
